In [0]:
import requests
import json
from pyspark.sql.types import *

response = requests.get("https://ghoapi.azureedge.net/api/NCD_BMI_18A?$filter=SpatialDimType%20eq%20%27Country%27%20and%20Dim1%20eq%20%27BTSX%27")
adultUnderweight = response.json()['value']
dfUnderweight = spark.createDataFrame(adultUnderweight, schema = StructType([StructField("SpatialDim", StringType(), True),
                                                                             StructField("TimeDim", StringType(), True),
                                                                             StructField("Dim1", StringType(), True),
                                                                             StructField("NumericValue", StringType(), True)]))

dfUnderweight = dfUnderweight.withColumn("TimeDim", dfUnderweight["TimeDim"].cast('int')) \
                             .withColumn("NumericValue", dfUnderweight["NumericValue"].cast('float')) \
                             .withColumnRenamed("TimeDim","Year") \
                             .withColumnRenamed("SpatialDim","Country") \
                             .withColumnRenamed("NumericValue","PercentUnderweightAdults") \
                             .drop("Dim1")

display(dfUnderweight)

In [0]:
# Make sure we have all the countries
dfUnderweight.select("Country").distinct().count()

Out[3]: 195

In [0]:
# KAFKA PRODUCER

def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))
        
#DO NOT DELETE THIS
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "who-underweight-final"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"

admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
# #ONLY NEEDS TO BE RUN ONCE
# topic_list = []

# topic_list.append(NewTopic(confluentTopicName, 1, 3))
# admin_client.create_topics(topic_list)
# futures = admin_client.create_topics(topic_list)


# try:
#     record_metadata = []
#     for k, future in futures.items():
#         # f = i.get(timeout=10)
#         print(f"type(k): {type(k)}")
#         print(f"type(v): {type(future)}")
#         print(future.result())

# except KafkaError:
#     # Decide what to do if produce request failed...
#     print(traceback.format_exc())
#     result = 'Fail'
# finally:
#     print("finally")

In [0]:
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

DF_Pandas = dfUnderweight.toPandas()

for i in range(len(DF_Pandas)):
    p.produce(confluentTopicName,DF_Pandas.iloc[i].to_json())
    p.flush()
    sleep(5)